In [1]:
from bs4 import BeautifulSoup
import lxml
import requests
import re
import pandas

## Idea

Could we use this data and then the data from wikipedia/imdb (ratings) to see if the episodes that Karen's is on are better than the typical episode?

In [104]:
url = 'https://theoffice.fandom.com/wiki/Mrs._California'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [105]:
# finding the next link
table = soup.find('section', class_ = 'pi-item pi-group pi-border-color')
# find the link to the next episode
next_ep = table.find_all('div', class_='pi-data-value pi-font')[-1].find(href=True)['href']

# find's a lot of good information
ele = soup.find_all('div', class_ = 'pi-data-value pi-font') 

# the recurring cast
page_uls = soup.find_all('ul')

ele[7].text

"""There's no good way to find the uls because we have no id so what I do is I go through all the ul's and see which
one has a previous element that has the text of 'recurring cast.' 
"""

for ul in page_uls:
    if ul.find_previous().text == 'Recurring cast':
        recur_cast_box = ul
        
page_uls[-12].find_previous().text
#dir(soup)

'Recurring cast'

In [130]:
# finding the second link which is the character's name
recur_cast_finder = re.compile("as (.+)")
recurring_cast = []

# for line in recur_cast_box.find_all("li"):
#     recurring_cast.append(recur_cast_finder.findall(line.text)[0]) # the 'a' tag finds all the links and we take the last (right-most) link in the line

for line in recur_cast_box.find_all("li"):
    line_clean = line.text.replace(u'\xa0', u' ')
    print(recur_cast_finder.findall(line_clean)[0])
    

Cathy Simms


In [127]:
x.replace(u'\xa0', u' ')

'Lindsey Broad as Cathy Simms'

In [121]:
recur_cast_finder.findall(x)

[]

In [113]:
recur_cast_finder.findall("Lindsey Broad as Cathy Simms")[0]

'Cathy Simms'

In [101]:
next_ep
recurring_cast

['Hank Tate', 'Nate']

In [132]:
base = 'https://theoffice.fandom.com'
url = 'https://theoffice.fandom.com/wiki/Pilot'
links = []
count = 0 # remove after tests

titles = []
seasons = []
dates = []
writers = []
directors = []

ep_recurring_cast = [] # this will be a list of lists

recur_cast_finder = re.compile("as (.+)")

while True:
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    # finding data
    ele = soup.find_all('div', class_ = 'pi-data-value pi-font')
    
    season = ele[0].text
    seasons.append(season)
    
    date = ele[3].text
    dates.append(date)
    
    writer = ele[4].text
    writers.append(writer)
    
    director = ele[5].text
    directors.append(director)
    
    title = ele[7].text
    titles.append(title)
    
    # find the recurring cast for the current episode
    recurring_cast = [] 
    
    page_uls = soup.find_all('ul') # we find the uls, one of them which is the box that has the recurring cast
    
    for ul in page_uls:
        if ul.find_previous().text == 'Recurring cast': # find the ul whose previous element's text is 'Recurring Cast'
            recur_cast_box = ul
            
            for line in recur_cast_box.find_all("li"):
                # it seems like even if the text of a line is broke down by tags in the html, we can still access the composite line text
                line_clean = line.text.replace(u'\xa0', u' ') # clean up some tex
                recurring_cast.append(recur_cast_finder.findall(line_clean)[0]) 

    # add the sub-list
    recurring_cast = ";".join(recurring_cast)
    ep_recurring_cast.append(recurring_cast)
        
    
    # finding the the next episode
    table = soup.find('section', class_ = 'pi-item pi-group pi-border-color')  # table that has a box
    try:
        next_ep = table.find_all('div', class_='pi-data-value pi-font')[-1].find(href=True)['href']  # find the last element in the box, and in that, find a link
    except TypeError: # when it hits Finale, it won't find a the href and it'll return an empty list which can't be subscripted and that'll return a type error
        break
    
    url = base + next_ep
    links.append(url)
    
# ## remove after tests       
#     count += 1
#     if count == 5:
#         break

In [133]:
titles

['Pilot',
 'Diversity Day',
 'Health Care',
 'The Alliance',
 'Basketball',
 'Hot Girl',
 'The Dundies',
 'Sexual Harassment',
 'Office Olympics',
 'The Fire',
 'Halloween',
 'The Fight',
 'The Client',
 'Performance Review',
 'Email Surveillance',
 'Christmas Party',
 'Booze Cruise',
 'The Injury',
 'The Secret',
 'The Carpet',
 'Boys and Girls',
 "Valentine's Day",
 "Dwight's Speech",
 'Take Your Daughter to Work Day',
 "Michael's Birthday",
 'Drug Testing',
 'Conflict Resolution',
 'Casino Night',
 'Gay Witch Hunt',
 'The Convention',
 'The Coup',
 'Grief Counseling',
 'Initiation',
 'Diwali',
 'Branch Closing',
 'The Merger',
 'The Convict',
 'A Benihana Christmas',
 'Back From Vacation',
 'Traveling Salesmen',
 'The Return',
 'Ben Franklin',
 "Phyllis' Wedding",
 'Business School',
 'Cocktails',
 'The Negotiation',
 'Safety Training',
 'Product Recall',
 "Women's Appreciation",
 'Beach Games',
 'The Job',
 'Fun Run',
 'Dunder Mifflin Infinity',
 'Launch Party',
 'Money',
 'Local A

In [134]:
import pandas as pd
import numpy as np

In [135]:
d = pd.DataFrame({"season":seasons,
                  "name":titles,
                  "dates":dates,
                  "writer":writers,
                  "directors":directors,
                  "recurring_cast":ep_recurring_cast})

In [146]:
d[~(d.recurring_cast == "")]

,season,name,dates,writer,directors,recurring_cast
0,Season 1,Pilot,"March 24, 2005",Ricky Gervais & Stephen Merchant and Greg Daniels,Ken Kwapis,Jan Levinson;Roy Anderson;Stanley Hudson;Kevin...
1,1,Diversity Day,"March 29, 2005",B.J. Novak,Ken Kwapis,Stanley Hudson;Kevin Malone;Meredith Palmer;Ke...
2,1,Health Care,"April 5, 2005",Paul Lieberstein,Ken Whittingham,Jan Levinson;Stanley Hudson;Kevin Malone;Mered...
3,1,The Alliance,"April 12, 2005",Michael Schur,Bryan Gordon,Roy Anderson;Stanley Hudson;Kevin Malone;Mered...
4,1,Basketball,"April 19, 2005",Greg Daniels,Greg Daniels,Darryl Philbin;Roy Anderson;Lonnie Collins;Jer...
...,...,...,...,...,...,...
162,8,Free Family Portrait Studio,"May 10, 2012",B.J. Novak,B.J. Novak,Robert Lipton;Ravi;David Wallace;Val;Nate;Teri...
163,9,New Guys,"September 20, 2012",Greg Daniels,Greg Daniels,Ryan Howard;Kelly Kapoor;Robert Lipton;David W...
164,9,Roy's Wedding,"September 27, 2012",Allison Silverman,Matt Sohn,Roy Anderson;Val;Bob Vance;Kenny Anderson
168,9,The Boat,"November 8, 2012",Dan Sterling,John Krasinski,Walter Bernard Jr.;Robert Lipton;David Wallace...


In [147]:
d.to_csv("the_office_data.csv")

In [148]:
d

,season,name,dates,writer,directors,recurring_cast
0,Season 1,Pilot,"March 24, 2005",Ricky Gervais & Stephen Merchant and Greg Daniels,Ken Kwapis,Jan Levinson;Roy Anderson;Stanley Hudson;Kevin...
1,1,Diversity Day,"March 29, 2005",B.J. Novak,Ken Kwapis,Stanley Hudson;Kevin Malone;Meredith Palmer;Ke...
2,1,Health Care,"April 5, 2005",Paul Lieberstein,Ken Whittingham,Jan Levinson;Stanley Hudson;Kevin Malone;Mered...
3,1,The Alliance,"April 12, 2005",Michael Schur,Bryan Gordon,Roy Anderson;Stanley Hudson;Kevin Malone;Mered...
4,1,Basketball,"April 19, 2005",Greg Daniels,Greg Daniels,Darryl Philbin;Roy Anderson;Lonnie Collins;Jer...
...,...,...,...,...,...,...
181,9,Stairmageddon,"April 11, 2013",Dan Sterling,Matt Sohn,
182,9,Paper Airplane,"April 25, 2013",Halsted Sullivan & Warren Lieberstein,Jesse Peretz,
183,9,Livin' the Dream,"May 2, 2013",Niki Schwartz-Wright,Jeffrey Blitz,
184,9,A.A.R.M,"May 9, 2013",Brent Forrester,David Rogers,
